# ISTM 6212 - Data Management for Analytics: Final Project
## Analyzing and exploring the FEC datasets with business analytics methods
### Group 7: Calvin Ji, Hrolfur Sveinsson, Peijia Wu, Sun Pil Howang, Yihang Zhao

___

We start by loading SQL and creating our database

In [1]:
%load_ext sql

In [2]:
!dropdb -U student finalproject

In [3]:
!createdb -U student finalproject

In [4]:
%sql postgresql://student@/finalproject

In [5]:
!pwd

/home/ubuntu


# Ask 1  (10%)
+ Use the FEC datasets. Collect on Amazon S3 the following datasets:
    + All Candidates and Candidate Master
    + Committee Master
    + Linkage
    + Contribution by individual to candidates
    + Contribution by Committees to candidates
    + Contribution from one committee to another
    + Years 2016 and 2020 (launch a new instance and increase the storage to 60 GB or more)
    
    
+  Acquire data and perform initial exploration to make sure it is suitable for dimensional modeling and analytical analysis

+ Expectation: all data files in a bucket. Make the bucket public so I can have access to it. Only one member of the group need to do this step. Document your bucket list, file sizes, date uploaded, link to the public directory

___

We use the data files from [FEC](https://www.fec.gov/data/browse-data/?tab=bulk-data)

The FEC data files are stored in our Amazon S3 [dmfa-2020-final-project-hs](https://s3.console.aws.amazon.com/s3/buckets/dmfa-2020-final-project-hs?region=us-east-1&tab=objects) bucket
+ Amazon resource name (ARN): arn:aws:s3:::dmfa-2020-final-project-hs

We acquire the data by using wget so we can start performing initial exploration to make sure that the data is suitable for dimensional modeling and analytical analysis. We start with the year 2020

In [6]:
!wget https://s3.amazonaws.com/dmfa-2020-final-project-hs/Allcandidates/weball20.txt

--2020-11-30 19:20:47--  https://s3.amazonaws.com/dmfa-2020-final-project-hs/Allcandidates/weball20.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.28.46
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.28.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 499472 (488K) [text/plain]
Saving to: ‘weball20.txt.1’

weball20.txt.1      100%[===================>] 487.77K  --.-KB/s    in 0.02s   

2020-11-30 19:20:47 (28.0 MB/s) - ‘weball20.txt.1’ saved [499472/499472]



In [7]:
!wget https://s3.amazonaws.com/dmfa-2020-final-project-hs/Candidatemaster/cn20.txt

--2020-11-30 19:20:47--  https://s3.amazonaws.com/dmfa-2020-final-project-hs/Candidatemaster/cn20.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.28.46
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.28.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 748825 (731K) [text/plain]
Saving to: ‘cn20.txt.1’

cn20.txt.1          100%[===================>] 731.27K  --.-KB/s    in 0.05s   

2020-11-30 19:20:48 (14.7 MB/s) - ‘cn20.txt.1’ saved [748825/748825]



In [8]:
!wget https://s3.amazonaws.com/dmfa-2020-final-project-hs/Candidatetocommitteelinkages/ccl20.txt

--2020-11-30 19:20:48--  https://s3.amazonaws.com/dmfa-2020-final-project-hs/Candidatetocommitteelinkages/ccl20.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.28.46
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.28.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 288476 (282K) [text/plain]
Saving to: ‘ccl20.txt.1’

ccl20.txt.1         100%[===================>] 281.71K  --.-KB/s    in 0.01s   

2020-11-30 19:20:48 (22.1 MB/s) - ‘ccl20.txt.1’ saved [288476/288476]



In [9]:
!wget https://s3.amazonaws.com/dmfa-2020-final-project-hs/Committeemaster/cm20.txt

--2020-11-30 19:20:48--  https://s3.amazonaws.com/dmfa-2020-final-project-hs/Committeemaster/cm20.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.28.46
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.28.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2184291 (2.1M) [text/plain]
Saving to: ‘cm20.txt.1’

cm20.txt.1          100%[===================>]   2.08M  --.-KB/s    in 0.07s   

2020-11-30 19:20:48 (29.7 MB/s) - ‘cm20.txt.1’ saved [2184291/2184291]



We convert all of the txt files into csv files

In [10]:
!csvformat -d "|" weball20.txt > all_candidates.csv

In [11]:
!csvformat -d "|" cn20.txt > candidate_master.csv

In [12]:
!csvformat -d "|" ccl20.txt > linkage.csv

In [13]:
!csvformat -d "|" cm20.txt > committee_master.csv

We check the number of rows in each dataset including headers

In [14]:
!wc -l *.csv

    3954 All_candidates.csv
    7757 Candidate_master.csv
   18061 Committee_master.csv
    7037 Linkage.csv
    3953 all_candidates.csv
    3954 all_candidates_header.csv
    7756 candidate_master.csv
    7757 candidate_master_header.csv
       1 ccl_header_file.csv
       1 cm_header_file.csv
       1 cn_header_file.csv
   18060 committee_master.csv
   18061 committee_master_header.csv
    7036 linkage.csv
    7037 linkage_header.csv
       0 weball_header_file.csv
  110426 total


After initial exploration, it looks like our data is suitable for dimensional modeling and analytical analysis. Next, we need to add a header row to each dataset and create our database tables using PostgreSQL

# Ask 2 (15%)
+ Acquire the data, survey it and wrangle it to suitable format

# You may use one or more of the following methods for acquiring and wrangling the data
+ Unix command line tools (e.g. csvkit)
+ Relational databases
+ Spark
+ Trifacta

# Describe any concerns with the data and changes you expect to overcome

# Pandas and R can only be used in visualization

# Expectations: document your findings in the code. Make sure you describe each step in a complete and concise manner

___

We acquire our header row data by using wget

In [15]:
!wget https://s3.amazonaws.com/dmfa-2020-final-project-hs/Allcandidates/weball_header_file.csv

--2020-11-30 19:20:50--  https://s3.amazonaws.com/dmfa-2020-final-project-hs/Allcandidates/weball_header_file.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.89.246
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.89.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 411 [text/csv]
Saving to: ‘weball_header_file.csv.1’

weball_header_file. 100%[===================>]     411  --.-KB/s    in 0s      

2020-11-30 19:20:50 (23.5 MB/s) - ‘weball_header_file.csv.1’ saved [411/411]



In [16]:
!wget https://s3.amazonaws.com/dmfa-2020-final-project-hs/Candidatemaster/cn_header_file.csv

--2020-11-30 19:20:50--  https://s3.amazonaws.com/dmfa-2020-final-project-hs/Candidatemaster/cn_header_file.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.89.246
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.89.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180 [text/csv]
Saving to: ‘cn_header_file.csv.1’

cn_header_file.csv. 100%[===================>]     180  --.-KB/s    in 0s      

2020-11-30 19:20:50 (9.91 MB/s) - ‘cn_header_file.csv.1’ saved [180/180]



In [17]:
!wget https://s3.amazonaws.com/dmfa-2020-final-project-hs/Candidatetocommitteelinkages/ccl_header_file.csv

--2020-11-30 19:20:50--  https://s3.amazonaws.com/dmfa-2020-final-project-hs/Candidatetocommitteelinkages/ccl_header_file.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.89.246
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.89.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79 [text/csv]
Saving to: ‘ccl_header_file.csv.1’

ccl_header_file.csv 100%[===================>]      79  --.-KB/s    in 0s      

2020-11-30 19:20:50 (4.81 MB/s) - ‘ccl_header_file.csv.1’ saved [79/79]



In [18]:
!wget https://s3.amazonaws.com/dmfa-2020-final-project-hs/Committeemaster/cm_header_file.csv

--2020-11-30 19:20:50--  https://s3.amazonaws.com/dmfa-2020-final-project-hs/Committeemaster/cm_header_file.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.89.246
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.89.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 158 [text/csv]
Saving to: ‘cm_header_file.csv.1’

cm_header_file.csv. 100%[===================>]     158  --.-KB/s    in 0s      

2020-11-30 19:20:50 (5.75 MB/s) - ‘cm_header_file.csv.1’ saved [158/158]



We use csvstack -H to create default headers (a,b,c,...) since the input csv file has no header row

In [19]:
!csvstack -H all_candidates.csv > all_candidates_header.csv

In [20]:
!csvstack -H candidate_master.csv > candidate_master_header.csv

In [21]:
!csvstack -H linkage.csv > linkage_header.csv

In [22]:
!csvstack -H committee_master.csv > committee_master_header.csv

Again, we check the number of rows in each dataset including headers. We do that to make sure we have added a header row for each dataset

In [23]:
!wc -l *.csv

    3954 All_candidates.csv
    7757 Candidate_master.csv
   18061 Committee_master.csv
    7037 Linkage.csv
    3953 all_candidates.csv
    3954 all_candidates_header.csv
    7756 candidate_master.csv
    7757 candidate_master_header.csv
       1 ccl_header_file.csv
       1 cm_header_file.csv
       1 cn_header_file.csv
   18060 committee_master.csv
   18061 committee_master_header.csv
    7036 linkage.csv
    7037 linkage_header.csv
       0 weball_header_file.csv
  110426 total


In [24]:
# Why does weball_header_file.csv contain zero rows?
# Does it matter, do we have to fix it?

Now we use csvstack to replace our default header rows (a,b,c,...) with our our header row files from the FEC datasets

In [25]:
!csvstack weball_header_file.csv all_candidates_header.csv > All_candidates.csv

In [26]:
!csvstack cn_header_file.csv candidate_master_header.csv > Candidate_master.csv

In [27]:
!csvstack ccl_header_file.csv linkage_header.csv > Linkage.csv

In [28]:
!csvstack cm_header_file.csv committee_master_header.csv > Committee_master.csv

Again, we check the number of rows in each dataset including headers. We do that to make sure we have finally added a header row that matches the column name of each column in each dataset

In [29]:
!wc -l *.csv

    3954 All_candidates.csv
    7757 Candidate_master.csv
   18061 Committee_master.csv
    7037 Linkage.csv
    3953 all_candidates.csv
    3954 all_candidates_header.csv
    7756 candidate_master.csv
    7757 candidate_master_header.csv
       1 ccl_header_file.csv
       1 cm_header_file.csv
       1 cn_header_file.csv
   18060 committee_master.csv
   18061 committee_master_header.csv
    7036 linkage.csv
    7037 linkage_header.csv
       0 weball_header_file.csv
  110426 total


In [30]:
# Why does weball_header_file.csv contain zero rows?
# Does it matter, do we have to fix it?

In [31]:
!csvcut -n Committee_master.csv

  1: CMTE_ID
  2: CMTE_NM
  3: TRES_NM
  4: CMTE_ST1
  5: CMTE_ST2
  6: CMTE_CITY
  7: CMTE_ST
  8: CMTE_ZIP
  9: CMTE_DSGN
 10: CMTE_TP
 11: CMTE_PTY_AFFILIATION
 12: CMTE_FILING_FREQ
 13: ORG_TP
 14: CONNECTED_ORG_NM
 15: CAND_ID


In [32]:
!head -n 1000 Committee_master.csv | csvstat

  1. "CMTE_ID"

	Type of data:          Text
	Contains null values:  False
	Unique values:         999
	Longest value:         9 characters
	Most common values:    C00000059 (1x)
	                       C00000422 (1x)
	                       C00000489 (1x)
	                       C00000547 (1x)
	                       C00000638 (1x)

  2. "CMTE_NM"

	Type of data:          Text
	Contains null values:  False
	Unique values:         999
	Longest value:         163 characters
	Most common values:    HALLMARK CARDS PAC (1x)
	                       AMERICAN MEDICAL ASSOCIATION POLITICAL ACTION COMMITTEE (1x)
	                       D R I V E POLITICAL FUND CHAPTER 886 (1x)
	                       KANSAS MEDICAL SOCIETY POLITICAL ACTION COMMITTEE (1x)
	                       INDIANA STATE MEDICAL ASSOCIATION POLITICAL ACTION COMMITTEE (1x)

  3. "TRES_NM"

	Type of data:          Text
	Contains null values:  False
	Unique values:         985
	Longest value:         30 characters
	Most common

In [33]:
%%sql
DROP TABLE IF EXISTS Committee_master;

CREATE TABLE Committee_master (
    CMTE_ID VARCHAR(9) NOT NULL,
    CMTE_NM VARCHAR(200),
    TRES_NM VARCHAR(90),
    CMTE_ST1 VARCHAR(34),
    CMTE_ST2 VARCHAR(34),
    CMTE_CITY VARCHAR(30),
    CMTE_ST VARCHAR(2),
    CMTE_ZIP VARCHAR(9),
    CMTE_DSGN VARCHAR(1),
    CMTE_TP VARCHAR(1),
    CMTE_PTY_AFFILIATION VARCHAR(3),
    CMTE_FILING_FREQ VARCHAR(1),
    ORG_TP VARCHAR(1),
    CONNECTED_ORG_NM VARCHAR(200),
    CAND_ID VARCHAR(9)
)

 * postgresql://student@/finalproject
Done.
Done.


[]

In [34]:
# There is a difference how the table is defined in Gproject2(Final version.ipynb and practice-w11-12.ipynb
# PRIMARY KEY (CMTE_ID)
# Does it matter, do we have to change our definition?

# The Null and Data type are taken from https://www.fec.gov/campaign-finance-data/committee-master-file-description/
# There is a difference how the Null and Data type are handled in Gproject2(Final version.ipynb and practice-w11-12.ipynb
# Does it matter, do we have to change some definitions?

Now we will load the data directly using the `COPY` command.  Note that this **requires** the use of an absolute path, so we adjust it to our location:

In [35]:
!pwd

/home/ubuntu


In [36]:
%%sql
SELECT *
FROM Committee_master;

 * postgresql://student@/finalproject
0 rows affected.


cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id


In [37]:
%%sql
COPY Committee_master FROM '/home/ubuntu/Committee_master.csv'
CSV
HEADER;

 * postgresql://student@/finalproject
18060 rows affected.


[]

In [38]:
%%sql
SELECT COUNT(*) FROM Committee_master;

 * postgresql://student@/finalproject
1 rows affected.


count
18060


In [39]:
!wc -l Committee_master.csv

18061 Committee_master.csv


In [40]:
%%sql
SELECT * FROM Committee_master
LIMIT 5;

 * postgresql://student@/finalproject
5 rows affected.


cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id
C00000059,HALLMARK CARDS PAC,SARAH MOE,2501 MCGEE,MD #500,KANSAS CITY,MO,64108,U,Q,UNK,M,C,None,None
C00000422,AMERICAN MEDICAL ASSOCIATION POLITICAL ACTION COMMITTEE,"WALKER, KEVIN MR.","25 MASSACHUSETTS AVE, NW",SUITE 600,WASHINGTON,DC,200017400,B,Q,None,M,M,DELAWARE MEDICAL PAC,None
C00000489,D R I V E POLITICAL FUND CHAPTER 886,JERRY SIMS JR,3528 W RENO,None,OKLAHOMA CITY,OK,73107,U,N,None,Q,L,None,None
C00000547,KANSAS MEDICAL SOCIETY POLITICAL ACTION COMMITTEE,JERRY SLAUGHTER,623 SW 10TH AVE,None,TOPEKA,KS,666121627,U,Q,UNK,Q,M,KANSAS MEDICAL SOCIETY,None
C00000638,INDIANA STATE MEDICAL ASSOCIATION POLITICAL ACTION COMMITTEE,"ACHENBACH, GRANT MR.","322 CANAL WALK, CANAL LEVEL",None,INDIANAPOLIS,IN,46202,U,Q,None,T,M,None,None


# Ask (6) (35%)

# Write queries to answer your analytical questions

# If you produce any charts, make sure to include your code or a copy of chart (if you use outside Jupyter notebook) 

# You are free to use any tool for visualization (e.g. tableau, ggplot, matplotlib,..)

# Expectation: set of queries, results and charts. Those should be compatible with your presentation.

___


In [41]:
%%sql
SELECT cmte_pty_affiliation, count(cmte_pty_affiliation)
FROM Committee_master
GROUP BY cmte_pty_affiliation
ORDER BY count(cmte_pty_affiliation) DESC
LIMIT 5;

 * postgresql://student@/finalproject
5 rows affected.


cmte_pty_affiliation,count
DEM,3103
REP,2907
IND,458
LIB,202
UNK,166


In [42]:
%%sql
SELECT org_tp, count(org_tp)
FROM Committee_master
GROUP BY org_tp
ORDER BY count(org_tp) DESC
LIMIT 5;

 * postgresql://student@/finalproject
5 rows affected.


org_tp,count
C,1671
T,726
M,478
L,341
W,95
